In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [5]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [6]:
# Load the data
file_path = Path('/Users/anitapereda/Documents/dataAnalytics/creditmodels/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [26]:
X = df.copy()
X = X.drop(['loan_status', 'home_ownership', 'verification_status', 'issue_d', 'loan_status',
        'pymnt_plan', 'initial_list_status', 'next_pymnt_d', 'application_type',
        'hardship_flag', 'debt_settlement_flag'], axis =1)

from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()

X_scaled = data_scaler.fit_transform(X)


In [24]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df['loan_status']= le.fit_transform(df['loan_status'])

y= df['loan_status'].ravel()


In [27]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X_scaled, y)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [38]:
# Resample the training data with the RandomOversampler
# YOUR CODE HERE
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler()

X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators= 100)
brf_model=brf.fit(X_train, y_train)


In [39]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

predictions= brf.predict(X_test)

acc_score = balanced_accuracy_score(y_test, predictions)

acc_score

0.702301117136924

In [35]:
# Display the confusion matrix
# YOUR CODE HERE
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,51,40
Actual 1,2887,14227


In [37]:
# Print the imbalanced classification report
# YOUR CODE HERE
classification_report_imbalanced(y_test, predictions)

'                   pre       rec       spe        f1       geo       iba       sup\n\n          0       0.02      0.56      0.83      0.03      0.68      0.45        91\n          1       1.00      0.83      0.56      0.91      0.68      0.48     17114\n\navg / total       0.99      0.83      0.56      0.90      0.68      0.48     17205\n'

In [42]:
importances = brf_model.feature_importances_
sorted(zip(brf_model.feature_importances_, X.columns), reverse=True)

[(0.06905064843485997, 'total_rec_prncp'),
 (0.06542582960996489, 'total_pymnt'),
 (0.06114794771419043, 'total_pymnt_inv'),
 (0.05961563131243783, 'last_pymnt_amnt'),
 (0.05301257622880518, 'total_rec_int'),
 (0.032925393257295175, 'int_rate'),
 (0.021279230636210227, 'mo_sin_old_rev_tl_op'),
 (0.019651909368156893, 'installment'),
 (0.01943296643163214, 'dti'),
 (0.0185857330957338, 'out_prncp_inv'),
 (0.017705535334405467, 'total_bal_ex_mort'),
 (0.017473558593430655, 'max_bal_bc'),
 (0.017451463340257038, 'revol_bal'),
 (0.01743747648916036, 'il_util'),
 (0.016771547999930397, 'out_prncp'),
 (0.01664083015420708, 'total_bal_il'),
 (0.01630695712276145, 'avg_cur_bal'),
 (0.01577524430750793, 'mo_sin_old_il_acct'),
 (0.015695396618900563, 'annual_inc'),
 (0.015329683436130476, 'mths_since_recent_inq'),
 (0.014873802902710152, 'total_bc_limit'),
 (0.014834190186609987, 'bc_util'),
 (0.014635546031763449, 'mths_since_rcnt_il'),
 (0.014442293625446622, 'tot_cur_bal'),
 (0.01436286340032

### Easy Ensemble AdaBoost Classifier

In [45]:
# Train the Classifier
# YOUR CODE HERE

from imblearn.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier()
eec_model = eec.fit(X_train, y_train)

y_pred= eec.predict(X_test)



In [46]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
acc_score = balanced_accuracy_score(y_test, y_pred)

acc_score


0.7273856504603262

In [48]:
# Display the confusion matrix
# YOUR CODE HERE
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,60,31
Actual 1,3501,13613


In [49]:
# Print the imbalanced classification report
# YOUR CODE HERE
classification_report_imbalanced(y_test, predictions)

'                   pre       rec       spe        f1       geo       iba       sup\n\n          0       0.02      0.56      0.84      0.04      0.69      0.46        91\n          1       1.00      0.84      0.56      0.91      0.69      0.49     17114\n\navg / total       0.99      0.84      0.56      0.91      0.69      0.49     17205\n'